In [1]:
import sys
import yaml
from sklearn.model_selection import train_test_split

sys.path.append('../../../')

In [2]:
from scripts.utils import load_data, preprocess_data_label_encoder, create_sequences, train_and_test, simulate_predictions

No GPU available. Running on CPU.


In [3]:
from src.models.bidir_lstm_multihead import build_model

In [4]:
YAML_FILE = '../../../configs/games/tic-tac-toe/bidir_lstm_multihead_50k.yml'

In [5]:
with open(YAML_FILE, 'r') as f:
    config = yaml.safe_load(f)
print(config)

{'name': '/games/tic-tac-toe/bidir_lstm_multihead_50k', 'model': {'type': 'Bidirectional Multihead LSTM'}, 'training': {'batch_size': 64, 'num_epochs': 10}, 'data': {'game': 'tic-tac-toe', 'sequence_length': 20, 'max_event_length': 50, 'path': '/games/tic-tac-toe/50k_single_agent.csv'}}


In [6]:
# Load and preprocess data
df = load_data(f'../../../data/processed/{config['data']['path']}')
df, e_event, e_agent, e_context = preprocess_data_label_encoder(df)

# Create sequences
sequence_length = config['data']['sequence_length']
X, y = create_sequences(df, sequence_length)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
model = train_and_test(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    e_event, 
    e_agent, 
    e_context, 
    name=config['name'], 
    epochs=config['training']['num_epochs'], 
    batch_size=config['training']['batch_size'],
    build_model=build_model
)

Epoch 1/10
6017/6017 ━━━━━━━━━━━━━━━━━━━━ 80s 13ms/step - agent_id_accuracy: 0.8110 - context_accuracy: 0.2399 - event_type_accuracy: 0.9126 - loss: 2.5986 - val_agent_id_accuracy: 0.9325 - val_context_accuracy: 0.2784 - val_event_type_accuracy: 0.9325 - val_loss: 2.1522
Epoch 2/10
6017/6017 ━━━━━━━━━━━━━━━━━━━━ 78s 13ms/step - agent_id_accuracy: 0.9316 - context_accuracy: 0.2845 - event_type_accuracy: 0.9321 - loss: 2.1322 - val_agent_id_accuracy: 0.9325 - val_context_accuracy: 0.3141 - val_event_type_accuracy: 0.9325 - val_loss: 2.0287
Epoch 3/10
6017/6017 ━━━━━━━━━━━━━━━━━━━━ 80s 13ms/step - agent_id_accuracy: 0.9341 - context_accuracy: 0.3087 - event_type_accuracy: 0.9345 - loss: 2.0232 - val_agent_id_accuracy: 0.9407 - val_context_accuracy: 0.3204 - val_event_type_accuracy: 0.9409 - val_loss: 1.9205
Epoch 4/10
6017/6017 ━━━━━━━━━━━━━━━━━━━━ 78s 13ms/step - agent_id_accuracy: 0.9422 - context_accuracy: 0.3237 - event_type_accuracy: 0.9422 - loss: 1.9021 - val_agent_id_accuracy: 0.9

Test Results:
loss: 1.5035
compile_metrics: 0.9722


In [8]:
simulate_predictions(
    data=X,
    model=model,
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Initial sequence:
MOVE, X, 1,2
MOVE, O, 2,0
MOVE, X, 2,2
MOVE, O, 1,0
GAME_END, system, O
GAME_START, system, New Game
MOVE, X, 2,2
MOVE, O, 2,0
MOVE, X, 0,2
MOVE, O, 0,0
MOVE, X, 1,2
GAME_END, system, X
GAME_START, system, New Game
MOVE, X, 1,1
MOVE, O, 0,1
MOVE, X, 1,2
MOVE, O, 2,1
MOVE, X, 2,2
MOVE, O, 0,2
MOVE, X, 0,0

Predicted moves:
Move 1: GAME_END, system, X
Move 2: GAME_START, system, New Game
Move 3: MOVE, X, 0,1
M